In [35]:
%%writefile ./src/elmtData.py

class Element:
    def __init__(self, id, number_of_nodes, material_set_id, node_ids=None):
        self._id = id
        self._material_set_id = material_set_id
        self._number_of_nodes = number_of_nodes
        if node_ids is None:
            self._node_ids = []
        else:
            self._node_ids = node_ids
    
    @property
    def id(self):
        return self._id
        
    @property
    def material_set_id(self):
        return self._material_set_id

    @property
    def number_of_nodes(self):
        return self._number_of_nodes

    @property
    def node_ids(self):
        return self._node_ids

    @node_ids.setter
    def node_ids(self, node_ids):
        if self._node_ids is None:
            self._node_ids = node_ids
        
class Node_2d:
    def __init__(self, nro, x_coord, y_coord):
        self._nro = nro
        self._x = x_coord
        self._y = y_coord

    @property
    def nro(self):
        return self._nro
    
    @property
    def coords(self):
        return [self._x, self._y]



Overwriting ./src/elmtData.py


## Read the Data

In [36]:
%%writefile qtree_to_vtk.py
import sys
import vtk
sys.path.append("./src/")
from elmtData import Node_2d, Element

list_of_nodes = []
list_of_elements = []

# Read elements from a file
# nro, nnodes, mat_nro, nodes
with open('./Output/mesh/selm.txt') as elmt_data:
    lines = elmt_data.readlines()
    for line in lines:
        elmt = list(map(int, line.split()))
        id, number_of_nodes, material_set_id, *node_ids = elmt
        node_ids = node_ids[:number_of_nodes]
        list_of_elements.append( Element(id, number_of_nodes, material_set_id, node_ids) )


Overwriting qtree_to_vtk.py


In [37]:
%%writefile -a qtree_to_vtk.py

# Read nodes from a file
# nro, x-coord, y-coord
with open('./Output/mesh/scor.txt') as node_data:
    lines = node_data.readlines()
    for line in lines:
        first, *rest = line.split()
        id = int(first)
        coords = list(map(float, rest))
        list_of_nodes.append( Node_2d(id, *coords) )

number_of_elements = len(list_of_elements)
number_of_nodes = len(list_of_nodes)

Appending to qtree_to_vtk.py


## Create and Fill the VTK Data Object with the Data 

In [38]:
%%writefile -a qtree_to_vtk.py

# Create VTK object for the data model
qtree_vtk_dataset = vtk.vtkUnstructuredGrid() 

# Create the points by defining their coordinates
points = vtk.vtkPoints()
for id in range(number_of_nodes):
    points.InsertPoint(id, [*list_of_nodes[id].coords, 0])
qtree_vtk_dataset.SetPoints(points)

# Create the cells by specifying connectivity
qtree_vtk_dataset.Allocate(number_of_elements)
for id in range(number_of_elements):
    point_ids = list(map(lambda a: a - 1, list_of_elements[id].node_ids))
    num_point_ids = list_of_elements[id].number_of_nodes
    qtree_vtk_dataset.InsertNextCell(7,num_point_ids,point_ids)

# Create data arrays (on cells)
array = vtk.vtkIntArray()
array.SetName("material_set_id")
array.SetNumberOfValues(number_of_elements)
for id in range(number_of_elements):
    array.SetValue(id, list_of_elements[id].material_set_id)
qtree_vtk_dataset.GetCellData().AddArray(array)

Appending to qtree_to_vtk.py


## Write The Data to Disk
Use the VTK writer for unstructured grids (`vtkXMLUnstructuredGridWriter`) to produce .vtu files

In [39]:
%%writefile -a qtree_to_vtk.py

writer = vtk.vtkXMLUnstructuredGridWriter()
writer.SetFileName("./Output/parv/QtreeMesh.vtu")
writer.SetInputData(qtree_vtk_dataset)
writer.Write()

print("Done!")

Appending to qtree_to_vtk.py


In [5]:
%run ./qtree_to_vtk.py

     Done!


In [8]:
%%writefile plot_qtree_mesh.py
from matplotlib.colors import ListedColormap 
import matplotlib.pyplot as plt

import pyvista
try: 
    mesh = pyvista.read('./Output/parv/QtreeMesh.vtu')
    pyvista.set_jupyter_backend('client')
    boring_cmap = plt.matplotlib.colormaps["rainbow"]
    mesh.plot(scalars='material_set_id', show_edges=True, cpos="xy", color=True, cmap=boring_cmap)
except FileNotFoundError:
    print("File is not found.")



Overwriting plot_qtree_mesh.py


In [6]:
qtree_grid = pyvista.UnstructuredGrid(qtree_vtk_dataset)
a = qtree_grid.plot(scalars='material_set_id', show_edges=True, cpos="xy", color=True, cmap=boring_cmap)

Widget(value="<iframe src='http://localhost:52594/index.html?ui=P_0x11c957bd0_0&reconnect=auto' style='width: …